In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
from pandarallel import pandarallel

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [3]:
ab = '../../lr_bulk/talon/human_talon_abundance.tsv'
filt_ab = '../../lr_bulk/cerberus/cerberus_filtered_abundance.tsv'
read_annot = '../../lr_bulk/talon/human_talon_read_annot.tsv'
t_metadata = '../../refs/cerberus/v40_transcript_metadata.tsv'
swan_file = '../../lr_bulk/cerberus/swan/swan_meta.p'
cerberus_h5 = '../../lr_bulk/cerberus/triplets/cerberus_annot_triplets.h5'
major_isos = '../../lr_bulk/cerberus/swan/major_isos.tsv'

ver = 'v40_cerberus'

min_tpm = 1
gene_subset = 'protein_coding'
obs_col = 'sample'

mouse_ab = '../../mouse/lr_bulk/talon/mouse_talon_abundance.tsv'
mouse_filt_ab = '../../mouse/lr_bulk/cerberus/cerberus_filtered_abundance.tsv'
mouse_read_annot = '../../mouse/lr_bulk/talon/mouse_talon_read_annot.tsv'
mouse_swan_file = '../../mouse/lr_bulk/swan/swan.p'
mouse_meta = '../../mouse/lr_bulk/lr_mouse_library_data_summary.tsv'
mouse_t_metadata = '../../mouse/refs/cerberus/vM25_transcript_metadata.tsv'
mouse_ver = 'vM25_cerberus'


## Distances between observed gene triplets and sample-level centroid gene triplets

In [4]:
ca = cerberus.read(cerberus_h5)
ca = get_centroids(ca, 
                   source='sample_det',
                   gene_subset=gene_subset,
                   ver=ver)
df = compute_dists(ca, 'sample_det_centroid', 'obs_det',
                   rm_1_iso_2=True,
                   gene_subset=gene_subset,
                   ver=ver)

/Users/fairliereese/miniconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:1300: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(js / 2.0)
/Users/fairliereese/miniconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:1300: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(js / 2.0)


In [5]:
n = len(df.index)
n_num = len(df.loc[df.z_score > 2].index)
print(f'{(n_num/n)*100:.2f}% genes ({n_num}/{n}) have z-score > 2 between obs. det and sample-level centroids')

3.18% genes (500/15734) have z-score > 2 between obs. det and sample-level centroids


In [10]:
n = len(df.loc[df.sector_obs_det=='tss'].index)
n_num = len(df.loc[(df.sector_obs_det=='tss')&(df.sector_sample_det_centroid!='tss')].index)
print(f'{(n_num/n)*100:.2f}% genes ({n_num}/{n}) in the obs_det TSS-high sector are not TSS-high for sample-level centroids')

94.83% genes (2129/2245) in the obs_det TSS-high sector are not TSS-high for sample-level centroids


In [12]:
# gene with the highest distance between obs_det and sample_det_centroid
df.sort_values(by='dist', ascending=False).head(1)['gname']

179    ACTA1
Name: gname, dtype: object